E01: train a trigram language model, i.e. take two characters as an input to predict the 3rd one. Feel free to use either counting or a neural net. Evaluate the loss; Did it improve over a bigram model?

In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

In [2]:
words = open('names.txt', 'r').read().splitlines()
print(words[:5])

['emma', 'olivia', 'ava', 'isabella', 'sophia']


In [3]:
import torch

In [15]:
N = torch.zeros((27, 27, 27), dtype=torch.int32)

In [16]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}

In [17]:
for w in words:
    chs = ['.']*2 + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        N[ix1, ix2, ix3] += 1

In [22]:
P = (N + 1).float()
P /= P.sum(2, keepdim=True) + 1e-6

In [23]:
g = torch.Generator().manual_seed(2147483647)

In [27]:
for _ in range(5):
    out = []
    ix1, ix2 = 0, 0  
    while True:
        p = P[ix1, ix2] 
        ix3 = torch.multinomial(p, num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix3])
        if ix3 == 0:  
            break
        ix1, ix2 = ix2, ix3  
    print(''.join(out))

daill.
jalantestian.
na.
sudaeveigh.
diren.


In [45]:
xs, ys = [], []
for w in words:
    chs = ['.']*2 + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        ix1 = stoi[ch1]
        ix2 = stoi[ch2]
        ix3 = stoi[ch3]
        xs.append((ix1, ix2))
        ys.append(ix3)
xs = torch.tensor(xs, dtype=torch.long)
ys = torch.tensor(ys, dtype=torch.long)
num = xs.shape[0]

xenc = torch.nn.functional.one_hot(xs, num_classes=27).float().view(num, -1)
W = torch.randn((54, 27), generator=g, requires_grad=True)

In [52]:
for k in range(100):
    logits = xenc @ W
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -torch.log(probs[torch.arange(num), ys]).mean()
    print(loss)

    W.grad = None
    loss.backward()
    W.data += -50 * W.grad



tensor(2.3861, grad_fn=<NegBackward0>)
tensor(2.3851, grad_fn=<NegBackward0>)
tensor(2.3842, grad_fn=<NegBackward0>)
tensor(2.3834, grad_fn=<NegBackward0>)
tensor(2.3825, grad_fn=<NegBackward0>)
tensor(2.3817, grad_fn=<NegBackward0>)
tensor(2.3809, grad_fn=<NegBackward0>)
tensor(2.3801, grad_fn=<NegBackward0>)
tensor(2.3794, grad_fn=<NegBackward0>)
tensor(2.3787, grad_fn=<NegBackward0>)
tensor(2.3780, grad_fn=<NegBackward0>)
tensor(2.3773, grad_fn=<NegBackward0>)
tensor(2.3766, grad_fn=<NegBackward0>)
tensor(2.3760, grad_fn=<NegBackward0>)
tensor(2.3754, grad_fn=<NegBackward0>)
tensor(2.3748, grad_fn=<NegBackward0>)
tensor(2.3742, grad_fn=<NegBackward0>)
tensor(2.3736, grad_fn=<NegBackward0>)
tensor(2.3731, grad_fn=<NegBackward0>)
tensor(2.3727, grad_fn=<NegBackward0>)
tensor(2.3725, grad_fn=<NegBackward0>)
tensor(2.3733, grad_fn=<NegBackward0>)
tensor(2.3762, grad_fn=<NegBackward0>)
tensor(2.3885, grad_fn=<NegBackward0>)
tensor(2.4001, grad_fn=<NegBackward0>)
tensor(2.4626, grad_fn=<N

In [53]:
for i in range(5): 
    out = []
  
    context = [0, 0]
    while True:
        xenc = torch.cat(
            [torch.nn.functional.one_hot(torch.tensor([ix]), num_classes=27).float() 
             for ix in context],
            dim=1
        )
        
        logits = xenc @ W
        counts = logits.exp()
        probs = counts / counts.sum(1, keepdim=True)

        ix = torch.multinomial(probs.squeeze(0), num_samples=1, replacement=True, generator=g).item()
        out.append(itos[ix])

        context = context[1:] + [ix]

        if ix == 0:
            break

    print(''.join(out))


camyn.
cor.
aryeshaumiylielyna.
aat.
raylaheree.


I don't know if it improved much. I'd say it is close.